In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kurtosis, skew, norm
from scipy.optimize import minimize
from numba import njit
from typing import Tuple, Optional
from enum import Enum
import qis
from dataclasses import fields, replace, asdict
from datetime import datetime, timedelta
import scipy.stats as ss
import copy
import scipy

from scipy.interpolate import splrep, BSpline
from numba.typed import List

# analytics
import sys
sys.path.insert(0,'../../') # just for jupyter  notebook
from stochvolmodels.pricers.hawkes_jd_pricer import HawkesJDParams, HawkesJDPricer, hawkesjd_chain_pricer, unpack_and_transform_pars_for_measure_change, unpack_pars

import tensorflow.experimental.numpy as tnp
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from stochvolmodels.data.test_option_chain import get_btc_test_chain_data, get_gld_test_chain_data_6m, get_sqqq_test_chain_data, get_spy_test_chain_data
from stochvolmodels.utils.funcs import to_flat_np_array, set_time_grid, timer, set_seed, transform_to_tfcomplex128, transform_from_tfcomplex128_to_np, slice_option_chain
from stochvolmodels.data.option_chain import OptionChain

import os
from stochvolmodels.pricers.core.bsm_pricer import infer_bsm_implied_vol, compute_bsm_price
import warnings
from stochvolmodels.MLE_estimator import hawkes_jd
from stochvolmodels.MLE_estimator import hawkes_jd_weekday

import pickle
warnings.filterwarnings('ignore')

2023-12-14 14:22:58.669739: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Read BTC spot
freq = 'D'

df = pd.read_csv('../../../resources/deribit/BTC-spot-combined.csv')
_id = pd.to_datetime(df.timestamp).apply(lambda x: x.hour) == 0

df = df.loc[_id, :]
df.timestamp = df.timestamp.apply(lambda x: x[:10])
df.timestamp = pd.to_datetime(df.timestamp)
price = df.spot
price.index = df.timestamp

returns = np.log(price/price.shift(1))[1:]

if freq == 'D':  
    dt = 1/365

elif freq == 'W':
    dt = 1/365*7

# Use BTC prior to option start day to calibrate for model parameters
_id = price.index <= options_start
MLE_estimator_daily = hawkes_jd.Hawkes_MLE_estimator(price.loc[_id])

In [ ]:
# Run POT
nu_p_grid = np.linspace(-.18, .18, 21)
nu_m_grid = np.linspace(-.18, .18, 21)

POT_results = []
for nu_p in nu_p_grid:
    for nu_m in nu_p_grid:
        if nu_p < nu_m:
            continue
        MLE_estimator_daily.run_Peak_over_Thresholds(nu_m, nu_p, verbose=False)
        POT_results.append({'nu_p0':nu_p, 'nu_m0':nu_m,
                            'nu_p':MLE_estimator_daily.nu_p, 'nu_m':MLE_estimator_daily.nu_m, 
                            'eta_p':MLE_estimator_daily.eta_p, 'eta_m':MLE_estimator_daily.eta_m,
                            'skew':ss.skew(MLE_estimator_daily.diffusion), 
                            'kurt':ss.kurtosis(MLE_estimator_daily.diffusion),
                            'n_pos_jumps':len(MLE_estimator_daily.positive_jumps_path), 
                            'n_neg_jumps':len(MLE_estimator_daily.negative_jumps_path), 
                            'percentage_diffusion':len(MLE_estimator_daily.diffusion)/len(MLE_estimator_daily.price)})
        
        
POT_results = pd.DataFrame(POT_results)
POT_results.loc[:,'error'] = (POT_results.loc[:, 'skew']**2) + (POT_results.loc[:, 'kurt'])**2
# print('daily nu_p:', MLE_estimator_daily.nu_p)
# print('daily nu_m:', MLE_estimator_daily.nu_m)

# diffusion obs: 0
# pos jumps obs: 0
# neg jumps obs: 1246
Skewness of diffusion obs: nan
Excess kurtosis of diffusion obs: nan
# diffusion obs: 0
# pos jumps obs: 0
# neg jumps obs: 1246
Skewness of diffusion obs: nan
Excess kurtosis of diffusion obs: nan
# diffusion obs: 0
# pos jumps obs: 0
# neg jumps obs: 1246
Skewness of diffusion obs: nan
Excess kurtosis of diffusion obs: nan
# diffusion obs: 0
# pos jumps obs: 0
# neg jumps obs: 1246
Skewness of diffusion obs: nan
Excess kurtosis of diffusion obs: nan
# diffusion obs: 0
# pos jumps obs: 1246
# neg jumps obs: 0
Skewness of diffusion obs: nan
Excess kurtosis of diffusion obs: nan
# diffusion obs: 0
# pos jumps obs: 0
# neg jumps obs: 1246
Skewness of diffusion obs: nan
Excess kurtosis of diffusion obs: nan
# diffusion obs: 0
# pos jumps obs: 0
# neg jumps obs: 1246
Skewness of diffusion obs: nan
Excess kurtosis of diffusion obs: nan
# diffusion obs: 0
# pos jumps obs: 1246
# neg jumps obs: 0
Skewness of diffusion obs: nan
Excess

In [ ]:
POT_results = POT_results.sort_values('error')
POT_results

,nu_p0,nu_m0,nu_p,nu_m,eta_p,eta_m,skew,kurt,n_pos_jumps,n_neg_jumps,percentage_diffusion,error
101,0.054,0.000,0.053331,-0.046207,0.031186,0.034054,0.001873,0.001281,87,113,0.838813,0.000005
99,0.054,-0.036,0.053331,-0.046144,0.031186,0.034117,0.001873,0.001281,87,113,0.838813,0.000005
98,0.054,-0.054,0.053322,-0.046107,0.031195,0.034154,0.001873,0.001281,87,113,0.838813,0.000005
97,0.054,-0.072,0.053318,-0.046244,0.031199,0.034017,0.001873,0.001281,87,113,0.838813,0.000005
111,0.072,-0.072,0.053288,-0.046184,0.030875,0.034078,-0.004940,-0.001159,88,113,0.838011,0.000026
...,...,...,...,...,...,...,...,...,...,...,...,...
226,0.180,0.108,5.355858,2.840149,NaN,2.837730,NaN,NaN,0,1246,0.000000,NaN
227,0.180,0.126,2.767929,2.713929,NaN,2.711509,NaN,NaN,0,1246,0.000000,NaN
228,0.180,0.144,2.767929,2.731929,NaN,2.729509,NaN,NaN,0,1246,0.000000,NaN
229,0.180,0.162,2.767929,2.749929,NaN,2.747509,NaN,NaN,0,1246,0.000000,NaN


In [ ]:
nu_p = POT_results.iloc[0].nu_p
nu_m = POT_results.iloc[0].nu_m

eta_p = POT_results.iloc[0].eta_p
eta_m = POT_results.iloc[0].eta_m

# Plug in calibrated nu_p and nu_m
MLE_estimator_daily.run_Peak_over_Thresholds(nu_m, nu_p, plug_in_nus=True)

# Calibrate mu and sigma
MLE_estimator_daily.calibrate_mu_and_sigma()
print('daily mu:', MLE_estimator_daily.mu, 'daily sigma:', MLE_estimator_daily.sigma)

In [ ]:
MLE_estimator_daily.calibrate_Hawkes_params(pars0 = (5, 400, 5, 400, 100, -100, 100, -100))
MLE_estimator_daily.Hawkes_results

In [ ]:
MLE_estimator_daily.calibrate_mu_and_sigma()
mu = MLE_estimator_daily.mu
sigma = MLE_estimator_daily.sigma

In [ ]:
# Retrieve calibration result
theta_p, kappa_p, theta_m, kappa_m, beta11, beta12, beta21, beta22 = MLE_estimator_daily.Hawkes_results.x

In [ ]:
# generalise intensities to the whole price path
lambdas_paths = returns.copy()
lambdas_paths.name = 'returns'
lambdas_paths = pd.DataFrame(lambdas_paths)
lambdas_paths.loc[:,'jump_sizes'] = 0

# assign jump sizes
_id = lambdas_paths.returns >= nu_p
lambdas_paths.jump_sizes[_id] = lambdas_paths.returns[_id]

# assign jump sizes
_id = lambdas_paths.returns <= nu_m
lambdas_paths.jump_sizes[_id] = lambdas_paths.returns[_id]

lambdas_paths.loc[:,'t'] = (lambdas_paths.index - lambdas_paths.index[0])
lambdas_paths.t = lambdas_paths.t.apply(lambda x: x.total_seconds()/SECONDS_PER_YEAR)

timestamps = pd.Series(lambdas_paths.index)


In [ ]:
lambda_p_left_arr = [theta_p]
lambda_m_left_arr = [theta_m]

lambda_p_right_arr = [theta_p]
lambda_m_right_arr = [theta_m]

for i in range(1, len(lambdas_paths)):
    _T = lambdas_paths.t.iloc[i-1]
    _lambda_p_T_right = lambda_p_right_arr[-1]
    _lambda_m_T_right = lambda_m_right_arr[-1]

    T = lambdas_paths.t.iloc[i]
    jump_size = lambdas_paths.jump_sizes.iloc[i]
    lambda_p_T_left  = (_lambda_p_T_right - theta_p)*np.exp(-kappa_p*(T - _T)) + theta_p
    lambda_m_T_left  = (_lambda_m_T_right - theta_m)*np.exp(-kappa_m*(T - _T)) + theta_m

    if jump_size > 0:
        lambda_p_T_right = lambda_p_T_left + beta11*jump_size
        lambda_m_T_right = lambda_m_T_left + beta21*jump_size
    else:
        lambda_p_T_right = lambda_p_T_left + beta12*jump_size
        lambda_m_T_right = lambda_m_T_left + beta22*jump_size

    lambda_p_left_arr.append(lambda_p_T_left)
    lambda_m_left_arr.append(lambda_m_T_left)

    lambda_p_right_arr.append(lambda_p_T_right)
    lambda_m_right_arr.append(lambda_m_T_right)

lambdas_paths.loc[:,'lambda_p_left'] = lambda_p_left_arr
lambdas_paths.loc[:,'lambda_m_left'] = lambda_m_left_arr

lambdas_paths.loc[:,'lambda_p_right'] = lambda_p_right_arr
lambdas_paths.loc[:,'lambda_m_right'] = lambda_m_right_arr

In [ ]:
lambdas_paths.loc[:,'theta_p'] = theta_p
lambdas_paths.loc[:,'theta_m'] = theta_m
lambdas_paths.loc[:,'kappa_p'] = kappa_p
lambdas_paths.loc[:,'kappa_m'] = kappa_m
lambdas_paths.loc[:,'beta11'] = beta11
lambdas_paths.loc[:,'beta12'] = beta12
lambdas_paths.loc[:,'beta21'] = beta21
lambdas_paths.loc[:,'beta22'] = beta22
lambdas_paths.loc[:,'eta_p'] = eta_p
lambdas_paths.loc[:,'eta_m'] = eta_m
lambdas_paths.loc[:,'nu_p'] = nu_p
lambdas_paths.loc[:,'nu_m'] = nu_m

In [ ]:
lambdas_paths.loc[:, 'mu'] = MLE_estimator_daily.mu
lambdas_paths.loc[:, 'sigma'] = MLE_estimator_daily.sigma

In [ ]:
lambdas_paths.to_csv('P_params.csv')